# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [175]:
import numpy as np
import pandas as pd

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [176]:
# Run this code:

salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [177]:
# Your code here:
print(salaries.dtypes)
print(salaries.shape)
print(salaries.head())

Name                  object
Job Titles            object
Department            object
Full or Part-Time     object
Salary or Hourly      object
Typical Hours        float64
Annual Salary        float64
Hourly Rate          float64
dtype: object
(33183, 8)
                  Name                              Job Titles  \
0    AARON,  JEFFERY M                                SERGEANT   
1      AARON,  KARINA   POLICE OFFICER (ASSIGNED AS DETECTIVE)   
2  AARON,  KIMBERLEI R                CHIEF CONTRACT EXPEDITER   
3  ABAD JR,  VICENTE M                       CIVIL ENGINEER IV   
4    ABASCAL,  REECE E             TRAFFIC CONTROL AIDE-HOURLY   

         Department Full or Part-Time Salary or Hourly  Typical Hours  \
0            POLICE                 F           Salary            NaN   
1            POLICE                 F           Salary            NaN   
2  GENERAL SERVICES                 F           Salary            NaN   
3       WATER MGMNT                 F           Salary

In [178]:
print(salaries.columns)

Index(['Name', 'Job Titles', 'Department', 'Full or Part-Time',
       'Salary or Hourly', 'Typical Hours', 'Annual Salary', 'Hourly Rate'],
      dtype='object')


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [179]:
print(salaries.isnull().sum())


Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [180]:
print(salaries['Salary or Hourly'].value_counts())


Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64


In [181]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [182]:
print(salaries['Department'].unique())

['POLICE' 'GENERAL SERVICES' 'WATER MGMNT' 'OEMC' 'CITY COUNCIL'
 'AVIATION' 'STREETS & SAN' 'FIRE' 'FAMILY & SUPPORT' 'PUBLIC LIBRARY'
 'TRANSPORTN' "MAYOR'S OFFICE" 'HEALTH' 'BUSINESS AFFAIRS' 'LAW' 'FINANCE'
 'CULTURAL AFFAIRS' 'COMMUNITY DEVELOPMENT' 'PROCUREMENT' 'BUILDINGS'
 'ANIMAL CONTRL' 'CITY CLERK' 'BOARD OF ELECTION' 'DISABILITIES'
 'HUMAN RESOURCES' 'DoIT' 'BUDGET & MGMT' 'TREASURER' 'INSPECTOR GEN'
 'HUMAN RELATIONS' 'COPA' 'BOARD OF ETHICS' 'POLICE BOARD' 'ADMIN HEARNG'
 'LICENSE APPL COMM']


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [183]:
from scipy.stats import ttest_1samp

In [184]:
salaries[salaries['Salary or Hourly']=='Hourly']['Hourly Rate'].mean()

32.78855771628024

In [185]:
salaries = salaries[salaries['Salary or Hourly'] == 'Hourly']
ttest_1samp(salaries['Hourly Rate'], 30)


Ttest_1sampResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92)

In [186]:
# The result of the statistical test is positive, which means that the average hourly wage is greater than $30. 
# With a p-value of 0.0, we can reject the null hypothesis that the average hourly wage is significantly different from $30.

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [187]:
salaries[salaries['Department']=='POLICE']['Hourly Rate'].mean()

9.384

Using the `crosstab` function, find the department that has the most hourly workers. 

In [188]:
common_dept = pd.crosstab(salaries['Department'], salaries['Salary or Hourly']).sort_values(by='Hourly', ascending=False).head(1)
print(common_dept)

Salary or Hourly  Hourly
Department              
STREETS & SAN       1862


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [189]:
salaries[salaries['Department']=='STREETS & SAN']['Hourly Rate'].mean()


33.72837808807734

In [190]:
salaries = salaries[salaries['Department'] == 'STREETS & SAN']
ttest_1samp(salaries['Hourly Rate'], 35)

Ttest_1sampResult(statistic=-9.567447887848152, pvalue=3.3378530564707717e-21)

In [191]:
# The result of the statistical test is negative, 
# which means that the mean of the hourly rate of the workers of the department of streets and sanitation is less than 35 dollars per hour.
# With a p-value of 0.0, we can reject the null hypothesis that the hourly wage is significantly different from $35.

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [192]:
from scipy.stats import sem
sem(salaries['Hourly Rate'])


0.13291129743573266

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [193]:
from scipy.stats import t
t.interval(0.95, len(salaries)-1, loc=np.mean(salaries['Hourly Rate']), scale=sem(salaries['Hourly Rate']))


(33.467707197668176, 33.98904897848651)

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [194]:
from statsmodels.stats.proportion import proportions_ztest
count = len(salaries[salaries['Salary or Hourly']=='Hourly'])
nobs = len(salaries)
value = 0.25
stat, pval = proportions_ztest(count, nobs, value)
print('{0:0.3f}'.format(pval))


0.000


C:\Users\gdlcm\AppData\Roaming\Python\Python310\site-packages\statsmodels\stats\weightstats.py:790: RuntimeWarning: divide by zero encountered in double_scalars
  zstat = value / std


In [195]:
# The resultt of the z test is zero, which means that the number of hourly workers in the city of Chicago is significantly different from 25% 
# at the 95% confidence level.